In [1]:
%run init.ipynb

C:\Users\Administrator\Anaconda3\lib\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.11) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [3]:
preprocessor = mz.models.DSSM.get_default_preprocessor(ngram_size=3)
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval:   0%| | 0/91 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.574 seconds.
Prefix dict has been built successfully.
Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 91/91 [00:02<00:00, 42.35it/s]
Processing text_right with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 96/96 [00:00<00:00, 204.37it/s]
Building Vocabulary from a datapack.: 100%|█████████████████████████████████████

In [4]:
preprocessor.context

{'embedding_input_dim': 299,
 'filter_unit': <mzcn.preprocessors.units.frequency_filter.FrequencyFilter at 0x18da91dbd68>,
 'ngram_process_unit': <mzcn.preprocessors.units.ngram_letter.NgramLetter at 0x18da91cacc0>,
 'ngram_vocab_size': 299,
 'ngram_vocab_unit': <mzcn.preprocessors.units.vocabulary.Vocabulary at 0x18da91e9e10>,
 'vocab_size': 299,
 'vocab_unit': <mzcn.preprocessors.units.vocabulary.Vocabulary at 0x18da91dbc18>}

In [5]:
triletter_callback = mz.dataloader.callbacks.Ngram(
    preprocessor, mode='aggregate')

trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=1,
    num_neg=4,
    callbacks=[triletter_callback]
)
devset = mz.dataloader.Dataset(
    data_pack=dev_pack_processed,
    callbacks=[triletter_callback]
)

In [12]:
padding_callback = mz.models.DSSM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback
)
devloader = mz.dataloader.DataLoader(
    dataset=devset,
    stage='dev',
    callback=padding_callback
)

In [8]:
model = mz.models.DSSM()

model.params['task'] = ranking_task
model.params['vocab_size'] = preprocessor.context['ngram_vocab_size']
model.params['mlp_num_layers'] = 3
model.params['mlp_num_units'] = 300
model.params['mlp_num_fan_out'] = 128
model.params['mlp_activation_func'] = 'relu'

model.build()

print(model, sum(p.numel() for p in model.parameters() if p.requires_grad))

DSSM(
  (mlp_left): Sequential(
    (0): Sequential(
      (0): Linear(in_features=299, out_features=300, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): ReLU()
    )
    (2): Sequential(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): ReLU()
    )
    (3): Sequential(
      (0): Linear(in_features=300, out_features=128, bias=True)
      (1): ReLU()
    )
  )
  (mlp_right): Sequential(
    (0): Sequential(
      (0): Linear(in_features=299, out_features=300, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): ReLU()
    )
    (2): Sequential(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): ReLU()
    )
    (3): Sequential(
      (0): Linear(in_features=300, out_features=128, bias=True)
      (1): ReLU()
    )
  )
  (out): Linear(in_features=1, out_features=1, bias=True)
) 6

In [13]:
optimizer = torch.optim.Adam(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=devloader,
    validate_interval=None,
    epochs=10
)

In [14]:
trainer.run()

[Iter-1 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2564 - normalized_discounted_cumulative_gain@5(0.0): 0.2564 - mean_average_precision(0.0): 0.2564



[Iter-2 Loss-0.983]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2564 - normalized_discounted_cumulative_gain@5(0.0): 0.2564 - mean_average_precision(0.0): 0.2564



[Iter-3 Loss-0.967]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2564 - normalized_discounted_cumulative_gain@5(0.0): 0.2564 - mean_average_precision(0.0): 0.2564



[Iter-4 Loss-1.051]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2564 - normalized_discounted_cumulative_gain@5(0.0): 0.2564 - mean_average_precision(0.0): 0.2564



[Iter-5 Loss-1.048]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2564 - normalized_discounted_cumulative_gain@5(0.0): 0.2564 - mean_average_precision(0.0): 0.2564



[Iter-6 Loss-0.964]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2564 - normalized_discounted_cumulative_gain@5(0.0): 0.2564 - mean_average_precision(0.0): 0.2564



[Iter-7 Loss-1.036]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2564 - normalized_discounted_cumulative_gain@5(0.0): 0.2564 - mean_average_precision(0.0): 0.2564



[Iter-8 Loss-0.970]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2564 - normalized_discounted_cumulative_gain@5(0.0): 0.2564 - mean_average_precision(0.0): 0.2564



[Iter-9 Loss-1.031]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2564 - normalized_discounted_cumulative_gain@5(0.0): 0.2564 - mean_average_precision(0.0): 0.2564



[Iter-10 Loss-0.973]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2564 - normalized_discounted_cumulative_gain@5(0.0): 0.2564 - mean_average_precision(0.0): 0.2564

Cost time: 4.173015832901001s


In [15]:
import gc
gc.collect()

2432